In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, concatenate, Dropout, Conv2D, GlobalAveragePooling2D,Conv2DTranspose, MaxPooling2D, UpSampling2D, Input, Reshape
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam, RMSprop
import numpy as np
import pandas as pd
import glob
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import cv2
%matplotlib inline
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from warnings import filterwarnings
from tensorflow.keras.models import load_model
import itertools
filterwarnings('ignore')
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import numpy as np
import os
import random


def set_seed(seed):
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)


seed = 42
set_seed(seed)

import tensorflow as tf

# Set the seed for reproducibility
tf.random.set_seed(42)

In [ ]:

# from tensorflow.keras.mixed_precision import experimental as mixed_precision
# policy = mixed_precision.Policy('float32')
# mixed_precision.set_policy(policy)
# print('Compute dtype: %s' % policy.compute_dtype)
# print('Variable dtype: %s' % policy.variable_dtype)


In [ ]:
train_dir = "train"
test_dir = "test"
val_dir= "val"


In [ ]:

input_shape = (224, 224)


In [ ]:
datagen_train = ImageDataGenerator(rescale=1./255,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  horizontal_flip=True,
                                  vertical_flip=False)



datagen_test = ImageDataGenerator(rescale=1./255)

datagen_val=ImageDataGenerator(rescale=1./255)

batch_size = 20
generator_train = datagen_train.flow_from_directory(directory=train_dir,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)

generator_val = datagen_val.flow_from_directory(directory=val_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                class_mode='categorical',
                                                  shuffle=False)
generator_test=datagen_test.flow_from_directory(directory=test_dir,
                                                  target_size=input_shape,
                                                 batch_size=batch_size,
                                                class_mode='categorical',
                                                 shuffle=False)

def print_layer_trainable():
    for layer in model1.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))


# print_layer_trainable()

In [ ]:
classes=list(generator_train.class_indices.keys())
class_count=len(classes)
print(class_count)

In [ ]:
import tensorflow as tf
from keras.layers import *

from tensorflow.keras import datasets, layers, models

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def squeeze_excite_block(input_tensor, ratio=16):
    channels = input_tensor.shape[-1]

    # Squeeze operation
    squeeze = layers.GlobalAveragePooling2D()(input_tensor)

    # Excite operation
    excite = layers.Dense(channels // ratio, activation='relu')(squeeze)
    excite = layers.Dense(channels, activation='sigmoid')(excite)

    # Reshape to (batch_size, 1, 1, channels)
    excite = tf.keras.layers.Reshape((1, 1, channels))(excite)

    # Scale the input with the excitation weights
    scaled_input = layers.Multiply()([input_tensor, excite])

    return scaled_input

# Define a more complex and parameter-efficient CNN model
input_shape = (224, 224, 3)
input_tensor = layers.Input(shape=input_shape)

x = layers.Conv2D(32, 3, activation='relu', padding='same')(input_tensor)
x = layers.BatchNormalization()(x)
x = squeeze_excite_block(x)

# Depthwise separable convolutional layers
x = layers.SeparableConv2D(64, 3, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = squeeze_excite_block(x)
x = layers.MaxPooling2D(2)(x)

x = layers.SeparableConv2D(128, 3, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = squeeze_excite_block(x)
x = layers.MaxPooling2D(2)(x)

x = layers.SeparableConv2D(128, 3, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = squeeze_excite_block(x)
x = layers.MaxPooling2D(2)(x)

x = layers.SeparableConv2D(256, 3, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = squeeze_excite_block(x)
x = layers.MaxPooling2D(2)(x)

# Global average pooling
x = layers.GlobalAveragePooling2D()(x)

# Fully connected layers
x = layers.Dense(256, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)

x = layers.Dense(128, activation='relu',dtype='float32')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)

# Output layer
output_tensor = layers.Dense(4, activation='softmax')(x)  #

# Create the model
model = models.Model(inputs=input_tensor, outputs=output_tensor)

# Compile the model
# model.compile(optimizer='adam',
#               loss='categorical_crossentropy',  # Use 'sparse_categorical_crossentropy' if your labels are integers
#               metrics=['accuracy'])

# Print the model summary
# model.summary()

In [ ]:
optimizer = Adam(lr=1e-4)
loss = 'categorical_crossentropy'
metrics = ['accuracy','Precision','Recall']

model.summary()
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

                                                                                                  
 multiply_3 (Multiply)          (None, 56, 56, 128)  0           ['batch_normalization_3[0][0]',  
                                                                  'reshape_3[0][0]']              
                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, 28, 28, 128)  0          ['multiply_3[0][0]']             
                                                                                                  
 separable_conv2d_3 (SeparableC  (None, 28, 28, 256)  34176      ['max_pooling2d_2[0][0]']        
 onv2D)                                                                                           
                                                                                                  
 batch_normalization_4 (BatchNo  (None, 28, 28, 256)  1024       ['separable_conv2d_3[0][0]']     
 rmalizati

In [ ]:
for i, layer in enumerate(model.layers):
    print(layer.name, i)

In [ ]:
model_layer_removed = tf.keras.models.Model(inputs=model.input, outputs=model.layers[32].output)
model_layer_removed.summary()

                                                                                                  
 multiply_3 (Multiply)          (None, 56, 56, 128)  0           ['batch_normalization_3[0][0]',  
                                                                  'reshape_3[0][0]']              
                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, 28, 28, 128)  0          ['multiply_3[0][0]']             
                                                                                                  
 separable_conv2d_3 (SeparableC  (None, 28, 28, 256)  34176      ['max_pooling2d_2[0][0]']        
 onv2D)                                                                                           
                                                                                                  
Total params: 70,550
Trainable params: 69,846
Non-trainable params: 704
_____________________________________

In [ ]:
x, y = generator_train.__next__()
x.shape

In [ ]:
predict_layers_removed = model_layer_removed.predict(x)

In [ ]:
predict_layers_removed.shape

In [ ]:
for i in range(128):
    plt.imshow(predict_layers_removed[0, :, :, i])
    plt.show()

In [ ]:
c,r=8,8
for i in predict_layers_removed :
    fig=plt.figure(figsize=(12,12))
    for j in range(1,c*r+1):
        fig=plt.subplot(r,c,j)
        fig.set_xticks([])
        fig.set_yticks([])
        plt.imshow(predict_layers_removed[0, :, :, j-1])
    plt.show()

In [ ]:
model_layer_removed = tf.keras.models.Model(inputs=model.input, outputs=model.layers[24].output)
x, y = generator_train.__next__()
predict_layers_removed = model_layer_removed.predict(x)
c,r=8,8
for i in predict_layers_removed :
    fig=plt.figure(figsize=(12,12))
    for j in range(1,c*r+1):
        fig=plt.subplot(r,c,j)
        fig.set_xticks([])
        fig.set_yticks([])
        plt.imshow(predict_layers_removed[0, :, :, j-1])

In [ ]:
model_layer_removed = tf.keras.models.Model(inputs=model.input, outputs=model.layers[8].output)
x, y = generator_train.__next__()
predict_layers_removed = model_layer_removed.predict(x)
c,r=8,8
for i in predict_layers_removed :
    fig=plt.figure(figsize=(12,12))
    for j in range(1,c*r+1):
        fig=plt.subplot(r,c,j)
        fig.set_xticks([])
        fig.set_yticks([])
        plt.imshow(predict_layers_removed[0, :, :, j-1])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

epochs =50
steps_per_epoch = generator_train.n / batch_size
steps_test = generator_test.n / batch_size

#steps_per_epoch = 20
# steps_test = 50
early_stop = EarlyStopping(patience=5)
history = model.fit_generator(generator=generator_train,
                              epochs=epochs,
                              steps_per_epoch=steps_per_epoch,
                              validation_data=generator_val,
                              validation_steps=steps_test,

                             )


Epoch 37/50
162/162 [==============================] - 22s 133ms/step - loss: 0.0548 - accuracy: 0.9818 - precision: 0.9821 - recall: 0.9806 - val_loss: 0.4170 - val_accuracy: 0.8875 - val_precision: 0.8870 - val_recall: 0.8826
Epoch 38/50
162/162 [==============================] - 22s 134ms/step - loss: 0.0514 - accuracy: 0.9803 - precision: 0.9821 - recall: 0.9788 - val_loss: 0.2807 - val_accuracy: 0.8973 - val_precision: 0.9129 - val_recall: 0.8973
Epoch 39/50
162/162 [==============================] - 22s 133ms/step - loss: 0.0503 - accuracy: 0.9815 - precision: 0.9827 - recall: 0.9809 - val_loss: 0.0162 - val_accuracy: 0.9976 - val_precision: 0.9976 - val_recall: 0.9976
Epoch 40/50
162/162 [==============================] - 22s 133ms/step - loss: 0.0421 - accuracy: 0.9880 - precision: 0.9883 - recall: 0.9868 - val_loss: 0.0416 - val_accuracy: 0.9780 - val_precision: 0.9780 - val_recall: 0.9780
Epoch 41/50
162/162 [==============================] - 22s 135ms/step - loss: 0.0329 - a

In [ ]:
model.save("GLNet.h5")

In [ ]:
from tensorflow.keras.utils import plot_model

# Plot the model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
#Get the accuracy score

test_score = model.evaluate_generator(generator_train)



print("[INFO] accuracy: {:.2f}%".format(test_score[1] * 100))

print("[INFO] Loss: {:.2f}%".format(test_score[0] * 100))

In [ ]:
#Get the accuracy score

test_score = model.evaluate_generator(generator_test)



print("[INFO] accuracy: {:.2f}%".format(test_score[1] * 100))

print("[INFO] Loss: {:.2f}%".format(test_score[0] * 100))

In [ ]:
classes=list(generator_train.class_indices.keys())
class_count=len(classes)
print(class_count)
li=[1,2,3,4]

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
Y_pred = model.predict_generator(generator_test)
y_pred = np.argmax(Y_pred, axis=1)
array = confusion_matrix(generator_test.classes, y_pred)
df_cm = pd.DataFrame(array, index =li,
                  columns = li)
plt.figure(figsize=(10,10))
sn.set(font_scale=2.0)
sn.heatmap(df_cm, annot=True, cmap='Blues', fmt='g')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title('Confusion matrix of GLNet')
plt.tight_layout()
# plt.savefig('CNN_confusion',dpi=200);
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(generator_test.classes, y_pred)

In [ ]:
target_names = []

for key in generator_train.class_indices:

    target_names.append(key)



print(target_names)

In [ ]:
preds = model.predict_generator(generator_test)
print(preds)

In [ ]:
print(classification_report(generator_test.classes, y_pred,digits=4))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 5))
ax = ax.ravel()

for i, met in enumerate(['precision', 'recall']):
    ax[i].plot(history.history[met])
    ax[i].plot(history.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].grid(color = '#e0e0eb')
    ax[i].legend(['train', 'val'])

In [ ]:

fig, ax = plt.subplots(1, 2, figsize=(20, 5))
ax = ax.ravel()
for i, met in enumerate([ 'accuracy', 'loss']):
    ax[i].plot(history.history[met])
    ax[i].plot(history.history['val_' + met])
    ax[i].set_title('GLNet Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].grid(color = '#e0e0eb')
    ax[i].legend(['train', 'val'],loc='best')
plt.show()

In [ ]:

plt.figure(figsize=(8, 5))
ax = plt.axes()
ax.set_facecolor("#eaeaf2")
plt.plot(history.history['accuracy'],label="train accuracy")
plt.plot(history.history['val_accuracy'],label="val accuracy")
plt.title('GLNet model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend()
plt.grid(color = 'w')
plt.savefig('CNN_accuracy_CNN_relu-test',dpi=200);
plt.show()


In [ ]:
plt.figure(figsize=(8, 5))

ax = plt.axes()
ax.set_facecolor("#eaeaf2")
plt.plot(history.history['loss'],label="train loss")
plt.plot(history.history['val_loss'],label="val loss")
plt.title('GLNet model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend()
plt.grid(color = 'w')
# plt.savefig('CNN_loss_CNN_relu-test',dpi=200);
plt.show()